# Module 2 - Working with Offline Feature Store 
## Features Store Dataset Extraction
**How to use Amazon SageMaker Feature Store to retrieve and share machine learning (ML) features in order to build feature sets that can be used for training a ML model.**

**Note:** Please set kernel to `Python 3 (Data Science)` and select instance to `ml.m5.4xlarge`

---

## Contents

1. [Overview](#Overview)
1. [Data Query](#Data-Query)
    1. [Look in S3 console](#Look-in-S3-console)
    1. [Query in Athena console](#Query-in-Athena-console)
    1. [Query using SageMaker SDK](#Query-using-SageMaker-SDK)
1. [Extract a feature set](#Extract-a-feature-set)    
1. [Row level time travel](#Row-Level-Time-Travel)
1. [Optional queries and validations](#Optional-queries-and-validations)
    1. [Browse the set of offline store files in the S3 console](#Browse-the-set-of-offline-store-files-in-the-S3-console.)
    1. [See the Glue tables that are used for Athena queries](#See-the-Glue-tables-that-are-used-for-Athena-queries.)
    1. [Examine contents of a sample offline store Parquet file](#Examine-contents-of-a-sample-offline-store-Parquet-file.)
    1. [Count the rows in an offline store](#Count-the-rows-in-an-offline-store.)
    1. [Get a random sample of offline store rows](#Get-a-random-sample-of-offline-store-rows.)

---

## Overview

In previous module (`Module-1 Introduction to SageMaker Feature Store`), we demonstrated how to create multiple features groups inside a Amazon SageMaker Feature Store and ingest data into it.

In this notebook, we will illustrate how to retrieve the ingested features from the multiple feature groups and combine them to build feature sets that can be used to train a ML model. We will cover the following aspects:

* Look at data via S3 console (Offline feature store)
* Athena query for dataset extraction - shown via Athena console
* Athena query for dataset extraction (programmatically using SageMaker SDK)
* Extract training dataset and persist to S3
* Some additional queries to visualize a Parquet file, count the rows and get a random sample of offline store

---


## Data Query

Before starting, in order to get into Feature Store in SageMaker Studio in the left side menu you need to select the SageMaker Components and registries, then select Feature Store from the list of components and click on the Feature store button.

![Feature Store](./images/FS_0.png "Feature Store")

First, let's start by looking at the feature store data. 

Once the features groups are created (as demonstrated in `Module-1`), we should be able to see the three features groups - `orders`, `products` and `customers` from the SageMaker Studio UI.

![Features Groups](./images/FS1.png "Features Groups")

Double click on one feature group to open a new correspondent tab in Studio. Inside one particular feature group, we can check the location of the offline data by looking for s3 on the bottom right like in the following image. 

![Orders FG](./images/FS2.png "Orders FG")

### Look in S3 console

In a different tab please open S3 console in your AWS account. The location of the offline feature store data is in S3 inside the default S3 bucket and prefix will respect the following format:

s3://DEFAULT_BUCKET/sagemaker-feature-store/ACCOUNT_ID/sagemaker/REGION/offline-store/

![S3 Location](./images/S3_1.png "S3 Location")

We notice that in S3 we have a timestamp suffix added for each feature group name and this corresponds to what is displayed in Athena console as tables.

You can get the S3 URL from your environment after launching the cell referenced by: [Browse the set of offline store files in the S3 console.](#Browse-the-set-of-offline-store-files-in-the-S3-console.)

Inside each feature group, we have a `data` subdirectory followed by directories partitioned by Parquet files.

![S3 Files](./images/S3_2.png "S3 Files")

You will can get a look into a sample Parquet file after launching the cell referenced by: [Examine contents of a sample offline store Parquet file](#Examine-contents-of-a-sample-offline-store-Parquet-file.)

---
### Query in Athena console

If it is for the first time we are launching Athena in AWS console we need to click on `Get Started` button and then before we run the first query we need to set up a query results location in Amazon S3. For simplicity, we can choose the same default SageMaker bucket that is used by Feature Store.

![Athena results location](./images/AthenaSetupMessage.png "Athena results location")

After setting the query results location, on the left panel we need to select the `AwsDataCatalog` as Data source and the `sagemaker_featurestore` as Database.

We can run now run a query for the offline feature store data in Athena. To select the entries from the orders feature group we use the following SQL query. You will need to replace the orders table name with the corresponded value from your environment.

```sql
select * from "<orders-feature-group-table-name>"
limit 20
```

![Athena Orders](./images/AthenaOrders.png "Athena Orders")

You can notice that Feature Store adds the `write_time`, `api_invocation_time` and `is_deleted` fields to the offline store.

Similarly, we can query the products and the customers feature group data by replacing the table name with the correspondent value. 

```sql
select * from "<products-feature-group-table-name>"
limit 20
select * from "<customers-feature-group-table-name>"
limit 20
```

Since our three features groups are related, we can do a join query by grouping all the information by product ID and by customer ID like here below. Please make sure you are replacing the table names with the corresponding values from your environment.

```sql
select *
FROM
  "<orders-feature-group-table-name>"
, "<products-feature-group-table-name>"  
, "<customers-feature-group-table-name>"
WHERE ("<orders-feature-group-table-name>"."customer_id" = "<customers-feature-group-table-name>"."customer_id")
AND ("<orders-feature-group-table-name>"."product_id" = "<products-feature-group-table-name>"."product_id")
limit 20
```

![Athena Join](./images/AthenaJoin.png "Athena Join")

---

### Query using SageMaker SDK

Extract the data from Feature Store using Athena query using SageMaker SDK

In [11]:
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker import get_execution_role
import sagemaker
import logging
import boto3
import pandas as pd
import time
import re
import os
import sys
sys.path.append('..')
from utilities import Utils
from utilities import feature_store_helper

In [12]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Initialize default parameters

In [13]:
sagemaker_session = sagemaker.Session()
account_id = sagemaker_session.account_id()
role = sagemaker.get_execution_role()
region = sagemaker_session.boto_region_name
default_bucket = sagemaker_session.default_bucket()
s3_client = boto3.client('s3', region_name=region)
query_results= 'sagemaker-featurestore-workshop'
prefix = 'sagemaker-feature-store'

#### Initialize boto3 runtime

In [14]:
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = sagemaker.Session(boto_session=boto_session, 
                                          sagemaker_client=sagemaker_client, 
                                          sagemaker_featurestore_runtime_client=featurestore_runtime)

#### Retrieve the orders, products and customers feature group names

In [15]:
# Retrieve FG names
%store -r customers_feature_group_name
%store -r products_feature_group_name
%store -r orders_feature_group_name
# Retrieve FG row count
%store -r customers_count
%store -r products_count
%store -r orders_count

customers_fg = FeatureGroup(name=customers_feature_group_name, sagemaker_session=feature_store_session)  
products_fg = FeatureGroup(name=products_feature_group_name, sagemaker_session=feature_store_session)
orders_fg = FeatureGroup(name=orders_feature_group_name, sagemaker_session=feature_store_session)

#### Build Athena join query to combine the 3 features groups - `customers`, `products` & `orders`

In [16]:
customers_query = customers_fg.athena_query()
customers_table = customers_query.table_name

products_query = products_fg.athena_query()
products_table = products_query.table_name

orders_query = orders_fg.athena_query()
orders_table = orders_query.table_name

In [17]:
query_string = f'SELECT * FROM "{customers_table}", "{products_table}", "{orders_table}" ' \
               f'WHERE ("{orders_table}"."customer_id" = "{customers_table}"."customer_id") ' \
               f'AND ("{orders_table}"."product_id" = "{products_table}"."product_id")'
%store query_string
query_string

Stored 'query_string' (str)


'SELECT * FROM "fscw_customers_05_22_21_52_1684792370", "fscw_products_05_22_21_52_1684792393", "fscw_orders_05_22_21_52_1684792420" WHERE ("fscw_orders_05_22_21_52_1684792420"."customer_id" = "fscw_customers_05_22_21_52_1684792370"."customer_id") AND ("fscw_orders_05_22_21_52_1684792420"."product_id" = "fscw_products_05_22_21_52_1684792393"."product_id")'

In [18]:
output_location = f's3://{default_bucket}/{query_results}/query_results/'
print(f'Athena query output location: \n{output_location}')

Athena query output location: 
s3://sagemaker-us-east-1-161615149547/sagemaker-featurestore-workshop/query_results/


#### Check to see if data is available in offline store.

In [19]:
# Before extracting the data we need to check if the feature store was populated
offline_store_contents = None
while offline_store_contents is None:    
    customers_total_record_count = Utils.get_historical_record_count(customers_feature_group_name)
    products_total_record_count = Utils.get_historical_record_count(products_feature_group_name)
    orders_total_record_count = Utils.get_historical_record_count(orders_feature_group_name)
    if customers_total_record_count >= customers_count and \
        products_total_record_count >= products_count and \
        orders_total_record_count >= orders_count:
        logger.info('[Features are available in Offline Store!]')
        offline_store_contents = orders_total_record_count
    else:
        logger.info('[Waiting for data in Offline Store ...]')
        time.sleep(60)

[Waiting for data in Offline Store ...]
[Waiting for data in Offline Store ...]
[Waiting for data in Offline Store ...]
[Waiting for data in Offline Store ...]
[Features are available in Offline Store!]
[Features are available in Offline Store!]


#### Run Athena query and load the output as a Pandas dataframe.

In [22]:
orders_query.run(query_string=query_string, output_location=output_location)
orders_query.wait()
joined_df = orders_query.as_dataframe()
joined_df.head()

,write_time,api_invocation_time,is_deleted,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,...,write_time.2,api_invocation_time.2,is_deleted.2,order_id,customer_id.1,product_id.1,purchase_amount,is_reordered,event_time.2,n_days_since_last_purchase
0,2023-05-22 21:59:28.189 UTC,2023-05-22 21:54:29.000 UTC,False,C631,0,0,2023-05-22T14:15:48.986Z,1,0,0,...,2023-05-22 22:00:57.766 UTC,2023-05-22 21:57:20.000 UTC,False,O15738,C631,P5903,0.204653,0,2023-05-22T14:23:25.587Z,0.844961
1,2023-05-22 21:59:28.189 UTC,2023-05-22 21:54:29.000 UTC,False,C631,0,0,2023-05-22T14:15:48.986Z,1,0,0,...,2023-05-22 22:00:57.888 UTC,2023-05-22 21:57:23.000 UTC,False,O15873,C631,P15299,0.186040,1,2023-05-22T14:23:25.596Z,0.387597
2,2023-05-22 21:59:28.189 UTC,2023-05-22 21:54:29.000 UTC,False,C631,0,0,2023-05-22T14:15:48.986Z,1,0,0,...,2023-05-22 22:01:09.803 UTC,2023-05-22 21:58:12.000 UTC,False,O11677,C631,P10488,0.777525,0,2023-05-22T14:23:25.330Z,0.197674
3,2023-05-22 21:59:28.189 UTC,2023-05-22 21:54:29.000 UTC,False,C631,0,0,2023-05-22T14:15:48.986Z,1,0,0,...,2023-05-22 22:01:09.536 UTC,2023-05-22 21:58:25.000 UTC,False,O93152,C631,P10632,0.890495,1,2023-05-22T14:23:31.551Z,0.100775
4,2023-05-22 21:59:28.189 UTC,2023-05-22 21:54:29.000 UTC,False,C631,0,0,2023-05-22T14:15:48.986Z,1,0,0,...,2023-05-22 22:01:09.811 UTC,2023-05-22 21:56:56.000 UTC,False,O39724,C631,P2673,0.881188,1,2023-05-22T14:23:27.299Z,0.945736


In [23]:
joined_df.columns

Index(['write_time', 'api_invocation_time', 'is_deleted', 'customer_id', 'sex',
       'is_married', 'event_time', 'age_18-29', 'age_30-39', 'age_40-49',
       'age_50-59', 'age_60-69', 'age_70-plus', 'n_days_active',
       'write_time.1', 'api_invocation_time.1', 'is_deleted.1', 'product_id',
       'event_time.1', 'category_baby_food_formula',
       'category_baking_ingredients', 'category_candy_chocolate',
       'category_chips_pretzels', 'category_cleaning_products',
       'category_coffee', 'category_cookies_cakes', 'category_crackers',
       'category_energy_granola_bars', 'category_frozen_meals',
       'category_hair_care', 'category_ice_cream_ice',
       'category_juice_nectars', 'category_packaged_cheese',
       'category_refrigerated', 'category_soup_broth_bouillon',
       'category_spices_seasonings', 'category_tea',
       'category_vitamins_supplements', 'category_yogurt', 'write_time.2',
       'api_invocation_time.2', 'is_deleted.2', 'order_id', 'customer_id.1'

## Extract a feature set

**Note:** This extracted feature set will be used for model training in `Module-3`

##### Drop columns which are not needed for training

In [24]:
model_df = joined_df.drop(['order_id', 
                           'customer_id', 
                           'product_id', 
                           'event_time', 
                           'write_time', 
                           'api_invocation_time', 
                           'is_deleted', 
                           'product_id.1', 
                           'event_time.1', 
                           'write_time.1', 
                           'api_invocation_time.1', 
                           'is_deleted.1', 
                           'customer_id.1', 
                           'purchase_amount',
                           'event_time.2', 
                           'n_days_since_last_purchase',
                           'write_time.2', 
                           'api_invocation_time.2', 
                           'is_deleted.2'], axis=1)

In [25]:
model_df.head(5)

,sex,is_married,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active,category_baby_food_formula,...,category_ice_cream_ice,category_juice_nectars,category_packaged_cheese,category_refrigerated,category_soup_broth_bouillon,category_spices_seasonings,category_tea,category_vitamins_supplements,category_yogurt,is_reordered
0,0,0,1,0,0,0,0,0,0.55411,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0.55411,0,...,0,0,0,0,0,0,0,1,0,1
2,0,0,1,0,0,0,0,0,0.55411,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0.55411,0,...,0,0,0,1,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,0.55411,0,...,0,0,0,0,0,0,0,0,0,1


#### Write transformed features to local `data` directory

In [26]:
model_df.to_csv('./data/train/transformed.csv', index=False)

#### Copy file from local to S3

In [28]:
boto3.Session().resource('s3').Bucket(default_bucket).Object(os.path.join(query_results, 'transformed.csv')).upload_file('./data/train/transformed.csv')

## Row Level Time Travel

Features in different feature groups can be updated at different times. In order to build a training dataset based on specific event timestamps, we need a way to extract the feature values across all the feature group which are valid at the given timestamp using point-in-time queries. Data scientists can query for the exact set of feature values that would have been available at a specific time, without the chance of including data from beyond that time.

[Refer to this blog post for details](https://aws.amazon.com/blogs/machine-learning/build-accurate-ml-training-datasets-using-point-in-time-queries-with-amazon-sagemaker-feature-store-and-apache-spark/)

![Row Level Time Travel](./images/m2_nb1_row_level_time_travel.png "Row Level Time Travel")


Here we use a helper class containing many utility functions. The `get_features` method implements row-level time travel for us, taking in a feature set across multiple feature groups, and taking in an events dataframe with timestamps and identifiers (join keys). In this simple example, our offline store only has a single version of the data, so this is a trivial example. In a real-world example, the event timestamps would be distributed across history, and the time travel function would accurately pull the features from each feature group based on those timestamps. Note that we can either specify an explicit list of individual features, or we can use a wildcard to retrieve all features from a given feature group. `get_features` is a convenient way to gather a training dataset without having to write complicated SQL queries.

Note - This is just demonstrating the use of row-level time travel, but the rest of the workshop does not depend upon the results.

In [29]:
#Building a dataframe of transformed order data
orders_df = pd.read_csv('./data/transformed/orders.csv')

multi_id_events = []

#Iterating over the first 10 rows from the data frame
for ind in orders_df.head(10).index:
    event = [orders_df['event_time'][ind], orders_df['order_id'][ind],
             orders_df['product_id'][ind], orders_df['customer_id'][ind]]
    multi_id_events.append(event)
    
multi_id_df = pd.DataFrame(multi_id_events, columns=['my_event_time', 'order_id', 'product_id', 'customer_id' ])
multi_id_df.head(10)





,my_event_time,order_id,product_id,customer_id
0,2023-05-22T14:23:24.376Z,O1,P8577,C8
1,2023-05-22T14:23:24.376Z,O2,P12984,C8510
2,2023-05-22T14:23:24.376Z,O3,P13498,C6316
3,2023-05-22T14:23:24.376Z,O4,P11612,C8392
4,2023-05-22T14:23:24.376Z,O5,P6096,C1816
5,2023-05-22T14:23:24.376Z,O6,P4821,C2551
6,2023-05-22T14:23:24.376Z,O7,P597,C7232
7,2023-05-22T14:23:24.376Z,O8,P10039,C9107
8,2023-05-22T14:23:24.376Z,O9,P10881,C1234
9,2023-05-22T14:23:24.377Z,O10,P6110,C2652


In [30]:
from utilities.feature_store_helper import FeatureStore
fs = FeatureStore()

# Calling the helper function to return the feature values across multiple feature groups valid for the given 
# event timestamps. Note the usage of wildcard for the product feature group.


fs.get_features(multi_id_df, 'my_event_time', 
                   features=[f'{customers_feature_group_name}:is_married',
                             f'{products_feature_group_name}:*',
                             f'{orders_feature_group_name}:purchase_amount'],
               parallel=False)


{'QueryExecution': {'QueryExecutionId': '46b46ebc-3f73-4ccf-8f3e-f2dd4d24a4fd', 'Query': 'DROP TABLE tmp_events_16847930680332115', 'StatementType': 'DDL', 'ResultConfiguration': {'OutputLocation': 's3://sagemaker-us-east-1-161615149547/offline-store/query_results/46b46ebc-3f73-4ccf-8f3e-f2dd4d24a4fd.txt'}, 'ResultReuseConfiguration': {'ResultReuseByAgeConfiguration': {'Enabled': False}}, 'QueryExecutionContext': {'Database': 'sagemaker_featurestore'}, 'Status': {'State': 'FAILED', 'StateChangeReason': '[ErrorCategory:USER_ERROR, ErrorCode:PERMISSION_ERROR], Detail:Amazon Athena experienced a permission error. Please provide proper permission and submitting the query again. If the issue reoccurs, contact AWS support for further assistance. You will not be charged for this query. We apologize for the inconvenience., Message:Amazon Athena experienced a permission error. Please provide proper permission and submitting the query again. If the issue reoccurs, contact AWS support for further

,my_event_time,order_id,product_id,customer_id,is_married,category_baby_food_formula,category_baking_ingredients,category_candy_chocolate,category_chips_pretzels,category_cleaning_products,...,category_ice_cream_ice,category_juice_nectars,category_packaged_cheese,category_refrigerated,category_soup_broth_bouillon,category_spices_seasonings,category_tea,category_vitamins_supplements,category_yogurt,purchase_amount
0,2023-05-22T14:23:24.376Z,O1,P8577,C8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.565446
1,2023-05-22T14:23:24.376Z,O2,P12984,C8510,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0.566832
2,2023-05-22T14:23:24.376Z,O3,P13498,C6316,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0.569802
3,2023-05-22T14:23:24.376Z,O4,P11612,C8392,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.924653
4,2023-05-22T14:23:24.376Z,O5,P6096,C1816,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.121188
5,2023-05-22T14:23:24.376Z,O6,P4821,C2551,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.402772
6,2023-05-22T14:23:24.376Z,O7,P597,C7232,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.797228
7,2023-05-22T14:23:24.376Z,O8,P10039,C9107,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0.129604
8,2023-05-22T14:23:24.376Z,O9,P10881,C1234,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.221188
9,2023-05-22T14:23:24.377Z,O10,P6110,C2652,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.398515


## Optional queries and validations

### Browse the set of offline store files in the S3 console.

In [31]:
customers_s3_console_url = Utils.get_offline_store_url(customers_feature_group_name)
products_s3_console_url = Utils.get_offline_store_url(products_feature_group_name)
orders_s3_console_url = Utils.get_offline_store_url(orders_feature_group_name)

logger.info('Review customers offline store partitioned data files here: '+customers_s3_console_url)
logger.info('\nReview products offline store partitioned data files here: '+products_s3_console_url)
logger.info('\nReview orders offline store partitioned data files here: '+orders_s3_console_url)

Review customers offline store partitioned data files here: https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-east-1-161615149547?region=us-east-1&prefix=sagemaker-feature-store/161615149547/sagemaker/us-east-1/offline-store/fscw_customers_05_22_21_52_1684792370/data/
Review customers offline store partitioned data files here: https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-east-1-161615149547?region=us-east-1&prefix=sagemaker-feature-store/161615149547/sagemaker/us-east-1/offline-store/fscw_customers_05_22_21_52_1684792370/data/

Review products offline store partitioned data files here: https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-east-1-161615149547?region=us-east-1&prefix=sagemaker-feature-store/161615149547/sagemaker/us-east-1/offline-store/fscw_products_05_22_21_52_1684792393/data/

Review products offline store partitioned data files here: https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-east-1-161615149547?region=us-east-1&prefix=sagemake

### See the Glue tables that are used for Athena queries.

In [20]:
customers_glue_console_url = Utils.get_glue_table_url(customers_feature_group_name)
products_glue_console_url = Utils.get_glue_table_url(products_feature_group_name)
orders_glue_console_url = Utils.get_glue_table_url(orders_feature_group_name)

logger.info('To see the customers Glue table that was created for you, go here: '+customers_glue_console_url)
logger.info('\nTo see the products Glue table that was created for you, go here: '+products_glue_console_url)
logger.info('\nTo see the orders Glue table that was created for you, go here: '+orders_glue_console_url)

To see the customers Glue table that was created for you, go here: https://console.aws.amazon.com/glue/home?region=us-east-1#table:catalog=161615149547;name=fscw_customers_05_22_15_24_1684769071;namespace=SageMaker_FeatureStore

To see the products Glue table that was created for you, go here: https://console.aws.amazon.com/glue/home?region=us-east-1#table:catalog=161615149547;name=fscw_products_05_22_15_24_1684769123;namespace=SageMaker_FeatureStore

To see the orders Glue table that was created for you, go here: https://console.aws.amazon.com/glue/home?region=us-east-1#table:catalog=161615149547;name=fscw_orders_05_22_15_24_1684769171;namespace=SageMaker_FeatureStore


### Examine contents of a sample offline store Parquet file.

In [32]:
# Let's have a look to the customers FG
customers_sample_filename = Utils.download_sample_offline_file(customers_feature_group_name)
logger.info('Downloaded sample Parquet file from offline store: '+customers_sample_filename+'\n')

customers_sample_df = pd.read_parquet(customers_sample_filename)
customers_sample_df.head()

Downloaded sample Parquet file from offline store: 20230522T141607Z_GdXMEaxYVnGCXAim.parquet

Downloaded sample Parquet file from offline store: 20230522T141607Z_GdXMEaxYVnGCXAim.parquet



,write_time,api_invocation_time,is_deleted,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,2023-05-22 15:12:32.102000+00:00,2023-05-22 15:07:33+00:00,False,C6,0,1,2023-05-22T14:15:48.019Z,1,0,0,0,0,0,0.130137
1,2023-05-22 15:12:32.102000+00:00,2023-05-22 15:07:33+00:00,False,C1879,0,0,2023-05-22T14:15:51.522Z,1,0,0,0,0,0,0.302055
2,2023-05-22 15:12:32.102000+00:00,2023-05-22 15:07:33+00:00,False,C1255,1,0,2023-05-22T14:15:50.117Z,0,0,0,1,0,0,0.046575
3,2023-05-22 15:12:32.102000+00:00,2023-05-22 15:07:33+00:00,False,C643,1,0,2023-05-22T14:15:49.005Z,0,0,0,0,0,1,0.529452
4,2023-05-22 15:12:32.102000+00:00,2023-05-22 15:07:33+00:00,False,C3755,1,1,2023-05-22T14:15:55.308Z,0,0,0,0,0,1,0.486986


In [33]:
import os
os.remove(customers_sample_filename)

### Count the rows in an offline store.

In [34]:
# Get the number of rows from customers FG
customers_total_record_count = Utils.get_historical_record_count(customers_feature_group_name)
logger.info(f'Found {customers_total_record_count:,d} records in "{customers_feature_group_name}" feature group.')

Found 10,000 records in "fscw-customers-05-22-21-52" feature group.
Found 10,000 records in "fscw-customers-05-22-21-52" feature group.


### Get a random sample of offline store rows.

In [35]:
# Get random rows from customers FG
customers_sample_df = Utils.sample(customers_feature_group_name, n=5)
customers_sample_df.head()

Running query :
 SELECT * FROM "fscw_customers_05_22_21_52_1684792370" tablesample bernoulli(25) limit 5
On  database: SageMaker_FeatureStore


,write_time,api_invocation_time,is_deleted,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,2023-05-22 21:59:28.183 UTC,2023-05-22 21:54:29.000 UTC,False,C4,1,1,2023-05-22T14:15:48.015Z,0,0,0,1,0,0,0.887671
1,2023-05-22 21:59:28.183 UTC,2023-05-22 21:54:31.000 UTC,False,C1301,1,0,2023-05-22T14:15:50.207Z,0,0,0,1,0,0,0.452055
2,2023-05-22 21:59:28.183 UTC,2023-05-22 21:54:31.000 UTC,False,C7538,0,1,2023-05-22T14:16:02.805Z,0,0,0,0,0,1,0.326712
3,2023-05-22 21:59:28.183 UTC,2023-05-22 21:54:31.000 UTC,False,C708,1,1,2023-05-22T14:15:49.107Z,0,0,0,0,1,0,0.543151
4,2023-05-22 21:59:28.183 UTC,2023-05-22 21:54:31.000 UTC,False,C4430,0,1,2023-05-22T14:15:56.751Z,1,0,0,0,0,0,0.368493
